# 計算ノードの追加

このnotebookでは、OCS OpenHPCテンプレートで作成されたSlurmクラスタに計算ノードを追加する手順について説明します。

## 前提

* 正常にジョブ実行ができる状態となっている状態を前提とします。構築途中であったり不具合のある状態の環境は対象としません。
* 「010-パラメータ設定.ipynb」で設定した最大ノード数を超えて計算ノードを追加することはできません。
* 「811-ノード数のスケジュール設定.ipynb」でスケジュール設定を行なっている場合は、このnotebookで計算ノードを追加することはできません。

## 準備

### UnitGroup名の指定

構築環境の UnitGroup名を指定します。

VCノードを作成時に指定した値を確認するために group_vars ファイル名の一覧を表示します。

In [ ]:
!ls -1 group_vars/*.yml | sed -e 's/^group_vars\///' -e 's/\.yml//' | sort

UnitGroup名を次のセルに指定してください。

In [ ]:
# (例)
# ugroup_name = 'OpenHPC'

ugroup_name = 

### group_varsの読み込み

次のセルを実行すると「010-パラメータの設定.ipynb」で指定したパラメータを読み込みます。読み込むパラメータの値は、UnitGroup名に指定した 値に対応するものになります。UnitGroup名の指定が誤っていると意図したパラメータが読み込めないので注意してください。

In [ ]:
%run scripts/group.py

gvars = load_group_vars(ugroup_name)

group_varsの内容を確認しておきます。

In [ ]:
!cat group_vars/{ugroup_name}.yml

指定されたUnitGroupでスケジュール設定が行われていないことを確認します。

In [ ]:
if gvars.get("vcnode_schedule") is not None:
    raise RuntimeError("スケジュール設定が行われています")

### VCCアクセストークンの入力

VCノードを起動するにはVC Controller(VCC)にアクセスして、操作を行う必要があります。VCCにアクセスするために必要となるアクセストークンをここで入力します。

In [ ]:
from getpass import getpass
vcc_access_token = getpass()

入力されたアクセストークンが正しいことを、実際にVCCにアクセスして確認します。

In [ ]:
from common import logsetting
from vcpsdk.vcpsdk import VcpSDK
vcp = VcpSDK(vcc_access_token)

上のセルの実行結果がエラーとなり以下のようなメッセージが表示されている場合は、入力されたアクセストークンに誤りがあります。

```
config vc failed: http_status(403)
2021/XX/XX XX:XX:XX UTC: VCPAuthException: xxxxxxx:token lookup is failed: permission denied
```

エラーになった場合はこの節のセルを全て `unfreeze` してから、もう一度アクセストークンの入力を行ってください。

> `unfreeze`するにはNotebookのツールバーにある`unfreeze below in section`ボタンなどを利用してください。

## 追加するノードのパラメータ設定

計算ノード追加の準備として、変数などの設定を実施します。

現在の計算ノード数を確認します。

In [ ]:
gvars["compute_nodes"]

計算ノードの最大ノード数を確認します。

In [ ]:
gvars["max_compute_nodes"]

追加するノード数を`nodes_to_add`に設定します。指定された値と現在のノード数を足した値が最大ノード数以下となる値を指定してください。

In [ ]:
# (例)
# nodes_to_add = 1

nodes_to_add = 1

追加する計算ノード数が適切かどうかチェックします。

In [ ]:
if not isinstance(nodes_to_add, int):
    msg = f'整数を指定してください: {nodes_to_add}'
    raise RuntimeError(msg)
if nodes_to_add < 1:
    msg = f'1以上の値を指定してください: {nodes_to_add}'
    raise RuntimeError(msg)
if gvars["compute_nodes"] + nodes_to_add > gvars["max_compute_nodes"]:
    msg = f'最大ノード数を超えています: {nodes_to_add}'
    raise RuntimeError(msg)

## mdx VMの起動

計算ノードとなるmdx VMを起動し、VCノードとして使用できる状態にします。mdx以外のクラウドを利用している場合は、この章をスキップして次章の[5.VCノードの追加](#VCノードの追加)から実行してください。

### mdx操作の準備

以下のセルを実行してmdx REST API認証トークンを入力します。

In [ ]:
from getpass import getpass
mdx_token = getpass("mdx API token")

mdx REST APIエンドポイントにIPv6で接続しようとすると到達不可となる場合があるため、以下のセルを実行してIPv4での接続を強制します。

In [ ]:
def use_ipv4_only():
    import socket
    old_getaddrinfo = socket.getaddrinfo
    def new_getaddrinfo(*args, **kwargs):
        responses = old_getaddrinfo(*args, **kwargs)
        return [response
                for response in responses
                if response[0] == socket.AF_INET]
    socket.getaddrinfo = new_getaddrinfo

use_ipv4_only()

VCP SDK mdx用プラグインモジュールを読み込みます。

In [ ]:
from common import logsetting
from vcpsdk.plugins.mdx_ext import MdxResourceExt
mdx = MdxResourceExt(mdx_token)

### 計算ノード用VMパラメータの設定

以下のセルを実行することで、計算ノードのVMのパラメータを作成します。作成するVMは既存の計算ノードと同じパラメータ設定となります。

In [ ]:
%run scripts/mdx_ops.py

# disk_size
if 'compute_root_size' in gvars:
    disk_size = gvars['compute_root_size']
else:
    disk_size = 40

# shared_key読み込み
import os
with open(os.path.expanduser(gvars['ssh_public_key_path'])) as f:
    shared_key = f.read()

c_spec = mdx_get_vm_spec(
    mdx,
    gvars['mdx_compute_pack_num'],
    gvars['compute_use_gpu'],
    disk_size,
    gvars['mdx_segment_id'],
    shared_key
)

2023-01-31のmdx REST APIの仕様変更により、VMのspecに`service_level`を指定しないとVMデプロイのリクエストがエラーとなるようになりました。このため、`service_level`を`guarantee`に指定して、明示的に起動保証VMとしてデプロイするよう指定します。

In [ ]:
from vcpsdk.plugins.mdx_ext import MDX_VM_SPEC_SCHEMA

if not 'service_level' in MDX_VM_SPEC_SCHEMA['properties']:
    MDX_VM_SPEC_SCHEMA['properties'].update(
        {'service_level': {'type': 'string'}}
    )

c_spec.update({'service_level': 'guarantee'})

VM内のユーザの初期パスワードを`initial_passwd`に設定します。

In [ ]:
initial_passwd = 'mdx additional cnode passwd'

### VMデプロイ

追加する計算ノードをmdx上にデプロイし、IPアドレスの変更を実施します。

VMのデプロイと設定に数分程度要するため、実行中のセルの経過時間を表示するライブラリjupyter-autotime機能を有効化します。

In [ ]:
!pip install --no-deps jupyter-autotime
%load_ext autotime

追加する計算ノードをデプロイし、外部からアクセスできるようになるまで待ちます。この処理には数分程度要します。

In [ ]:
%run scripts/mdx_ops.py

mdx_deploy_vms(mdx, list(c_add_etc_hosts.values()), c_spec,
               project=gvars['mdx_project_name'], verbose=True)

VMに対し初期パスワードを設定します。

In [ ]:
%run scripts/mdx_ops.py

mdx_set_init_passwd(mdx, c_add_etc_hosts.values(),
                   gvars['ssh_private_key_path'],
                   initial_passwd)

IPアドレスを変更します。

In [ ]:
%run scripts/mdx_ops.py

mdx_change_ipaddrs(mdx, c_add_etc_hosts,
                   os.path.expanduser(gvars['ssh_private_key_path']),
                   verbose=True)

### VCノード向け設定

mdx VMに対し、VCノードとして使用できるよう設定します。

In [ ]:
%run scripts/mdx_ops.py

vcppubkey = vcp.get_publickey()
mdx_init_vcp(list(c_add_etc_hosts.keys()),
             os.path.expanduser(gvars['ssh_private_key_path']),
            vcppubkey)

## VCノードの追加

VcpSDKを呼び出しVCノードの追加を行います。

追加するVCノードのIPアドレスを確認します。

In [ ]:
hosts = gvars["compute_etc_hosts"]
current_nodes = gvars["compute_nodes"]
new_nodes = gvars["compute_nodes"] + nodes_to_add
ip_addresses = [x for x in hosts][current_nodes:new_nodes]
ip_addresses

VCノードを追加します。

In [ ]:
ug = vcp.get_ugroup(gvars["ugroup_name"])
ut = ug.get_unit("compute")
ut.add_nodes(ip_addresses=ip_addresses)

VCノードの状態を確認します。

In [ ]:
ut.df_nodes()

## Slurmの状態を確認する

Slurmクラスタのノードの状態を確認します。

Slurmの状態を確認し、追加ノードが正しく追加されていることを確認します。

In [ ]:
!ansible {ugroup_name}_master -a sinfo

`srun`コマンドで、`hostname` コマンドを計算ノードで実行させてみます。

In [ ]:
new_nodes = gvars["compute_nodes"] + nodes_to_add
!ansible {ugroup_name}_master -a 'srun -l -N {new_nodes} hostname'

## notebook環境側のデータ更新

後でさらにノードを追加したり、ノードを削除する場合に備えて、ノードを追加した状態の設定に合わせて、noetbook環境側のデータを更新します。

### インベントリファイルの更新

ansibleのインベントリファイルに追加したノードの情報を追加します。

In [ ]:
import yaml
from copy import deepcopy
from pathlib import Path

inventory_yml = Path("inventory.yml")
with inventory_yml.open() as f:
    inventory = yaml.safe_load(f)
new_inventory = deepcopy(inventory)
new_inventory["all"]["children"][ugroup_name]["children"][f"{ugroup_name}_compute"]["hosts"].update({
    x: {} for x in ip_addresses
})
with inventory_yml.open(mode="w") as f:
    yaml.safe_dump(new_inventory, stream=f)

!cat {inventory_yml}

sshのknown_hostsを更新します。

In [ ]:
for addr in ip_addresses:
    !ssh-keygen -R {addr}
    !ssh-keyscan -H {addr} >> ~/.ssh/known_hosts

ansibleで到達可能か確認します。

In [ ]:
!ansible {ugroup_name}_compute -m ping

### group_varsの更新

group_varsファイルにの計算ノード数`compute_nodes`を更新します。

In [ ]:
%run scripts/group.py
compute_nodes_new = gvars["compute_nodes"] + nodes_to_add
update_group_vars(
    ugroup_name,
    compute_nodes=compute_nodes_new,
)

group_varsの内容を確認します。

In [ ]:
!cat group_vars/{ugroup_name}.yml

## その他

このnotebookでは扱いませんが、計算ノードに対してパッケージの追加や設定ファイルの更新などを実施している場合は、別途追加した計算ノードにも同じ設定が必要です。